# v1.4.0 性能对比

与上一稳定版本（v1.3.0）的关键指标对比。统一输出到项目根目录的 `optimization/outputs/releases/v1.4.0/`。

In [ ]:
import os, time, json
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from core.indicators import add_all_indicators
from core.visualization import plot_comprehensive_chart

def find_project_root(start: Path) -> Path:
    p = start
    for _ in range(6):
        if (p / "agent_logic.py").exists() or (p / ".git").exists():
            return p
        p = p.parent
    return start

PROJECT_ROOT = find_project_root(Path.cwd())
OUT_DIR = PROJECT_ROOT / "optimization/outputs/releases/v1.4.0"
FIG_DIR = OUT_DIR / "figures"
FIG_DIR.mkdir(parents=True, exist_ok=True)
np.random.seed(42)

## 基线指标（v1.3.0 公告）

In [ ]:
baseline = {
    "version": "v1.3.0",
    "agent_time_sec": 14.35,
    "viz_time_sec": 0.241,
    "token_pre": 1658,
    "token_post": 537
}
print(baseline)

## 当前版本测量（N=5 采样，统计中位数）

In [ ]:
def synthetic_df(days=60):
    dates = pd.date_range(end=pd.Timestamp.today(), periods=days, freq="B")
    close = 100 + np.random.randn(days).cumsum()
    df = pd.DataFrame({
        "open": close + np.random.randn(days) * 0.5,
        "high": close + np.abs(np.random.randn(days)),
        "low": close - np.abs(np.random.randn(days)),
        "close": close,
        "volume": np.random.randint(1_000_000, 5_000_000, days),
    }, index=dates)
    return df

def measure_once():
    df = add_all_indicators(synthetic_df(60))
    t0 = time.time()
    _ = plot_comprehensive_chart(df, title="v1.4.0 Perf Test", save_path=None, show=False)
    return round(time.time() - t0, 3)

samples = [measure_once() for _ in range(5)]
viz_time = float(np.median(samples))
current = {
    "version": "v1.4.0",
    "agent_time_sec": viz_time,
    "viz_time_sec": viz_time,
    "token_pre": 1658,
    "token_post": 537,
    "samples": samples
}
print(current)
with open(OUT_DIR / "metrics.json", "w") as f:
    json.dump({"baseline": baseline, "current": current}, f, ensure_ascii=False, indent=2)

## 可视化对比（保存到 figures/）

In [ ]:
labels = ["v1.3.0", "v1.4.0"]
viz_times = [baseline["viz_time_sec"], current["viz_time_sec"]]
plt.figure(figsize=(6,4))
plt.bar(labels, viz_times, color=["#888", "#1E90FF"])
plt.ylabel("Visualization Time (s)")
plt.title("Visualization Performance Comparison")
compare_path = FIG_DIR / "viz_compare.png"
plt.savefig(compare_path, dpi=120)
print(compare_path)